In [ ]:
import numpy as np
from pyJHTDB import libJHTDB
import pyJHTDB
import ctypes

In [ ]:
# toy cutout of big channel
'''
params = {"dataset": "channel5200", "function": "u", "output_filename": "channel",
          "stridet": 1, "stridex": 1, "stridey": 1, "stridez": 1, "ts": 1, "te": 11, "filter_width": 1,
          "token": "gov.sandia.dsdeigh-59a9eb2a", "xe": 10, "xs": 1, "ye": 10, "ys": 1,
          "ze": 10, "zs": 1}
''';

In [ ]:
'''
# bigger filter
params = {"dataset": "channel", "function": "u", "output_filename": "channel",
          "stridet": 1, "stridex": 20, "stridey": 20, "stridez": 20, "ts": 1, "te": 4000, "filter_width": 40,
          "token": "gov.sandia.dsdeigh-59a9eb2a", "xe": 2048, "xs": 1, "ye": 512, "ys": 1,
          "ze": 1536, "zs": 1}
''';

In [ ]:
# no filter
params = {"dataset": "channel", "function": "u", "output_filename": "channel",
          "stridet": 1, "stridex": 20, "stridey": 20, "stridez": 20, "ts": 1, "te": 4000, "filter_width": 1,
          "token": "gov.sandia.dsdeigh-59a9eb2a", "xe": 2048, "xs": 1, "ye": 512, "ys": 1,
          "ze": 1536, "zs": 1}

In [ ]:
# big channel
'''
params = {"dataset": "channel5200", "function": "u", "output_filename": "channel",
          "stridet": 1, "stridex": 20*5, "stridey": 20*3, "stridez": 20*5, "ts": 1, "te": 11, "filter_width": 20*4,
          "token": "gov.sandia.dsdeigh-59a9eb2a", "xe": 2048*5, "xs": 1, "ye": 512*3, "ys": 1,
          "ze": 1536*5, "zs": 1}
''';

Big Channel Dimensions: 10240 × 1536 × 7680

In [ ]:
auth_token=params["token"]
tstart=int(params.get("ts"))
tend=int(params.get("te"))
tstep=int(params.get("stridet"))
xstart=int(params.get("xs"))
ystart=int(params.get("ys"))
zstart=int(params.get("zs"))
xend=int(params.get("xe"))
yend=int(params.get("ye"))
zend=int(params.get("ze"))
xstep=int(params.get("stridex",1))
ystep=int(params.get("stridey",1))
zstep=int(params.get("stridez",1))
Filter_Width=int(params.get("filter_width",1))
time_step=int(params.get("stridet",0))
fields=params.get("function","u")
data_set=params.get("dataset","isotropic1024coarse")
output_filename=params.get("output_filename",data_set)

In [ ]:
import math
dims = (xend//xstep, yend//ystep, zend//zstep, tend//tstep, 3)
total_bytes = math.prod(dims)*4 # x4 for double byte size
total_GBs = total_bytes/1e9
print(total_GBs)

Big Channel Dimensions: 10240 × 1536 × 7680

In [ ]:
print(params)

In [ ]:
import os
try: os.mkdir('turbulence_f1_output')
except: pass
os.chdir('turbulence_f1_output')

## Direct Approach

In [ ]:
'''
lJHTDB = libJHTDB()
lJHTDB.initialize()
lJHTDB.add_token(auth_token)

result = lJHTDB.getbigCutout(
            data_set=data_set, fields=fields, t_start=tstart, t_end=tend, t_step=tstep,
            start=np.array([xstart, ystart, zstart], dtype = int),
            end=np.array([xend, yend, zend], dtype = int),
            step=np.array([xstep, ystep, zstep], dtype = int),
            filter_width=Filter_Width,filename=f'{output_filename}')
print(f'result.shape={result.shape}')
lJHTDB.finalize()
''';

## Indirect Approach

In [ ]:
#papermill_description=Downloading
%tb
lJHTDB = libJHTDB()
lJHTDB.initialize()
#lJHTDB.lib.turblibSetExitOnError(ctypes.c_int(1));
lJHTDB.add_token(auth_token)

tstep=1

import numpy as np

for t in range(1, 4001): #reversed(np.linspace(1, 4000, num=24, dtype=int)):
    fn = f'{output_filename}_t={t}'
    if os.path.exists(f'{fn}.h5'):
        if os.path.getsize(f'{fn}.h5')>100: continue
        else: 
            os.remove(f'{fn}.h5')
            os.remove(f'{fn}.xmf')
    tend=tstart=t

    print(f'downloading: {fn}')

    ## "filename" parameter is the file names of output files, if filename='N/A', no files will be written. 
    ##             For example, if filename='results', the function will write "results.h5" and "results.xmf".
    ## The function only returns the data at the last time step within [t_start:t_step:t_end]
    ## The function only returns the data in the last field. For example, result=p if field=[up].
    result = lJHTDB.getbigCutout(
            data_set=data_set, fields=fields, t_start=tstart, t_end=tend, t_step=tstep,
            start=np.array([xstart, ystart, zstart], dtype = int),
            end=np.array([xend, yend, zend], dtype = int),
            step=np.array([xstep, ystep, zstep], dtype = int),
            filter_width=Filter_Width,filename=f'{output_filename}_t={t}')
    print(f'result.shape={result.shape}')
lJHTDB.finalize()

In [ ]:
os.getcwd()